In [ ]:
import torch
from torch import nn
import numpy as np
import os
import imageio
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from dataset import get_rays
from model import Voxels
from rendering import rendering

In [ ]:
datapath = 'fox'
batch_size = 1024

o, d, target_px_values = get_rays(datapath,'train')
test_o, test_d, test_target_px_values = get_rays(datapath,'test')
dataloader = DataLoader(torch.cat((torch.from_numpy(o).reshape(-1,3),torch.from_numpy(d).reshape(-1,3),torch.from_numpy(target_px_values).reshape(-1,3)),dim=1),
                        batch_size=batch_size,shuffle=True)

In [ ]:
from tqdm import tqdm
from rendering import rendering
import torch

def training(model, optimizer, scheduler, tn, tf, nb_bins, nb_epochs, data_loader, device='cpu'):
    
    training_loss = []
    for epoch in (range(nb_epochs)):
        for batch in tqdm(data_loader):
            o = batch[:, :3].to(device)
            d = batch[:, 3:6].to(device)
            
            target = batch[:, 6:].to(device)
            
            prediction = rendering(model, o, d, tn, tf, nb_bins=nb_bins, device=device)
            
            loss = ((prediction - target)**2).mean()
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            training_loss.append(loss.item())
            
        scheduler.step()
        
        torch.save(model.cpu(), 'model_nerf')
        model.to(device)
        
    return training_loss

In [ ]:
device = 'cpu'
model = Voxels(scale=3,device = device)
nb_epochs = 15
nb_bins = 100
lr = 1e-3
gamma = 0.5
optimizer = torch.optim.Adam(model.parameters(),lr = lr)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones=[5,10], gamma = gamma)
tn = 8
tf = 12
n_bins = 100

training = training (model,optimizer,scheduler,tn,tf,nb_bins,nb_epochs,dataloader,device=device)